In [256]:
# Get libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [257]:
# Parse data file and post-process
filename = "data/jp_2020_name_vote.csv"

df = pd.read_csv(filename)

df.drop(columns=df.columns[11],axis=1,inplace=True)
df.drop(columns=df.columns[0:2],axis=1,inplace=True)
col_names = [(orig,re.search("\[.*\]",orig).group(0).strip("[]")) for orig in df.columns]
rename_cols = dict()
rename_cols.update(col_names)
df.rename(columns=rename_cols, inplace=True)
df = df.applymap(lambda x: int(x[-1]))

ranked_votes = df.to_numpy()
num_voters = len(df.index)
num_candidates = len(df.columns)
required_majority = int(np.ceil(num_voters/2))
remaining_contenders = df.columns
contenders = remaining_contenders

print("Number of voters: ", num_voters)
print("Number of candidates: ", num_candidates)
print("Required votes to win: ", required_majority)
df

Number of voters:  26
Number of candidates:  9
Required votes to win:  13


,Argonauts,Sea dragons,Axolotls,Coelacanths,Pteropods,Tardigrades,Ctenophores,Dinoflagellates,Antarctic Intermediate Water (AAIW)
0,2,4,1,3,6,7,5,8,9
1,2,9,8,1,5,7,3,6,4
2,3,2,6,1,7,4,5,8,9
3,1,8,6,5,2,3,7,4,9
4,2,6,9,1,4,7,3,5,8
5,7,4,9,5,2,8,1,3,6
6,3,4,8,1,2,6,5,7,9
7,5,4,2,7,3,6,1,8,9
8,4,2,9,5,3,8,7,1,6
9,7,9,6,8,4,3,5,2,1


In [258]:
# Helper functions
def get_weakest_candidate(df):
    """Returns the weakest candidate and the associated number of first-preference votes"""
    possible_weakest = df.columns
    
    # Check weakest by decreasing rank until only 1 is left
    for rank in range(1,len(df.columns)+1):
        possible_weakest = possible_weakest[(df[possible_weakest]==rank).sum()==(df[possible_weakest]==rank).sum().min()]
        if len(possible_weakest) == 1:
            break
            
    if len(possible_weakest) != 1:
        print("Something went wrong: there are ", len(possible_weakest), " weakest candidates.")
        
    final_votes = (df[possible_weakest]==1).sum()
    
    return (possible_weakest, final_votes)

def redistribute_votes(df, weakest_candidate):
    """Returns updated dataframe by dropping weakest candidate and shifting corresponding votes"""
    # drop weakest candidate, shift all ranks of effected candidates of effected voters
    df_updated = df.drop(weakest_candidate[0], axis=1)
    df_updated[df_updated.apply(lambda col: col > df[weakest_candidate[0]])] -= 1
    return df_updated

In [259]:
# Do Run-off voting
df_running = df.copy()

for round in range(num_candidates):
    print("="*10)
    print("Round: ", round+1)
    #print("Votes: ", df_running)
    weakest_candidate, final_votes = get_weakest_candidate(df_running)
    print("Weakest candidate: ", weakest_candidate[0])
    print("Final first-preference votes: ", final_votes[0])
    df_running = redistribute_votes(df_running, weakest_candidate)
    #print("Current first-preference standings:\n",(df_running==1).sum())
    if ((df_running==1).sum() > required_majority).any():
        break

print("="*10)
print("Finalist votes: \n", ((df_running==1).sum()).to_string())
print("Winner: ", ((df_running==1).sum()).idxmax())


Round:  0
Weakest candidate:  Axolotls
Final first-preference votes:  1
Round:  1
Weakest candidate:  Sea dragons
Final first-preference votes:  1
Round:  2
Weakest candidate:  Tardigrades
Final first-preference votes:  1
Round:  3
Weakest candidate:  Pteropods
Final first-preference votes:  1
Round:  4
Weakest candidate:  Ctenophores
Final first-preference votes:  2
Round:  5
Weakest candidate:  Dinoflagellates
Final first-preference votes:  5
Round:  6
Weakest candidate:  Antarctic Intermediate Water (AAIW)
Final first-preference votes:  7
Finalist votes: 
 Argonauts      15
Coelacanths    11
Winner:  Argonauts
